# Membership Inference Attack Analysis Examples

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/privML/privacy-evaluator/blob/team1sprint4/notebooks/membership_inference_attack_analysis.ipynb"><img src="https://raw.githubusercontent.com/privML/privacy-evaluator/team1sprint4/notebooks/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/privML/privacy-evaluator/blob/team1sprint4/notebooks/membership_inference_attack_analysis.ipynb"><img src="https://raw.githubusercontent.com/privML/privacy-evaluator/team1sprint4/notebooks/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

## Overview

In this notebook, we want to show you how to use the `privacy-evaluator` tool to perform a Membership Inference Attack Analysis on either a PyTorch or Tensorflow model. As an example we will conduct the Membership Inference Black Box Rule-Based Attack and apply an in-depth analysis on this method. To get a more general introduction into the  Membership Inference Attacks, please see [membership_inference_attack.ipynb](https://github.com/privML/privacy-evaluator/blob/team1sprint4/notebooks/membership_inference_attack.ipynb).

## Setup

First, you should set the notebook's runtime to use a GPU (e.g. if Colab is used go to ***Runtime > Change runtime type > Hardware accelerator***). Now we can install the `privacy-evaluator` package and import all needed modules.

In [ ]:
!pip3 install git+https://github.com/privML/privacy-evaluator@team1sprint4

In [ ]:
import tensorflow as tf
import torch
import numpy as np

import privacy_evaluator.models.torch.dcti.dcti as torch_dcti
import privacy_evaluator.models.tf.dcti.dcti as tf_dcti 

from privacy_evaluator.datasets.tf.cifar10 import TFCIFAR10
from privacy_evaluator.datasets.torch.cifar10 import TorchCIFAR10

from privacy_evaluator.classifiers.classifier import Classifier

from privacy_evaluator.attacks.membership_inference import MembershipInferenceAttackAnalysis
from privacy_evaluator.attacks.membership_inference import MembershipInferenceBlackBoxRuleBasedAttack

from privacy_evaluator.attacks.membership_inference.data_structures.attack_input_data import AttackInputData
from privacy_evaluator.attacks.membership_inference.data_structures.slicing import Slicing

## Conduct Membership Inference Attack Analysis

Now we can start with conducting the Membership Inference Attacks Analysis. Therefore, we prepared two similar paths: one for the PyTorch model and one for the TensorFlow model. For both paths, we use simple neural networks trained on the CIFAR-10 dataset and implement a Lightweight Deep Convolutional Neural Network architecture (for more details, please read the following paper: https://www.scitepress.org/Papers/2018/67520/67520.pdf).

### PyTorch

We start the analysis with the PyTorch model.

#### Load CIFAR10 Dataset

Before we can start to conduct the membership inference attack analysis, we need to load the dataset. The CIFAR10 dataset needs to be preprocessed in a specific manner to work for the PyTorch model.

In [ ]:
# Load CIFAR10 dataset as numpy array
x_train, y_train, x_test, y_test = TorchCIFAR10.numpy()

#### Prepare target model

Now, we need to initialize our pre-trained Lightweight Deep Convolutional Neural Network (short DCTI) as a generic `Classifier`. Therefore we need to specify the loss function used to train the model (in our case the `torch.nn.CrossEntropyLoss`), the number of classes and the input shape of our CIFAR-10 dataset.

In [ ]:
# Initalize PyTorch model as a Classifier
target_model = Classifier(
    torch_dcti.load_dcti(), # PyTorch DCTI 
    loss=torch.nn.CrossEntropyLoss(reduction="none"), # Loss function of the PyTorch model
    nb_classes=TorchCIFAR10.N_CLASSES, # Number of classes of the CIFAR10 dataset
    input_shape=TorchCIFAR10.INPUT_SHAPE # Input shape of the CIFAR10 dataset
)

#### Perpare attack analysis

Next, we prepare our attack analysis. To initialize our attack analysis we define the Membership Inference Attack method we want to perform (in this case we use the `MembershipInferenceBlackBoxRuleBasedAttack`) and the Attack Input Data. The Attack Input Data consists of two different sets. The first contains the data (`x_train`) and its corresponding labels (`y_train`) which were used to train the target model. The second contains the data (`x_test`) and its corresponding labels (`y_test`) which were not part of the training process of the target model.

In [ ]:
analysis = MembershipInferenceAttackAnalysis(
    MembershipInferenceBlackBoxRuleBasedAttack, 
    AttackInputData(
        x_train[:100], 
        y_train[:100], 
        x_test[:100], 
        y_test[:100]
    )
)

#### Define the slicing

Now we can define the slicing for our analysis. The slicing defines how the data will be sliced. Each slice will then be analysed separately. 

In [ ]:
slicing = Slicing(
    entire_dataset=True, 
    by_class=True, 
    by_classification_correctness=True
)

#### Perform Membership Inference Attack Analysis

Finally, we can perform our Membership Inference Attack Analysis. Therefore, we input the target model, the data that should be analysed, the membership labels (i.e. the labels which correctly describe if a data point is a member of the training dataset or not) and the splicing specification into the `analyse()` method. As a result, we get for each slice the indices of the corresponding data points, a human-readable description of the slice and the advantage score of the Membership Inference Attack (for more details about the advantage score, please read the following paper: https://arxiv.org/abs/1709.01604).

In [ ]:
result = analysis.analyse(
    target_model, 
    np.concatenate((x_train[:100], x_test[:100])), 
    np.concatenate((y_train[:100], y_test[:100])), 
    np.concatenate((np.ones(len(x_train[:100])), np.zeros(len(x_test[:100])))), 
    slicing
)

print("\n".join((str(r) for r in result)))

### TensorFlow

Now we do the same with the TensorFlow model.

#### Load CIFAR10 Dataset

Again, before we can start to conduct the membership inference attack analysis, we need to load the dataset. The CIFAR10 dataset needs to be preprocessed in a specific manner to work for the TensorFlow model.

In [ ]:
# Load CIFAR10 dataset as numpy array
x_train, y_train, x_test, y_test = TFCIFAR10.numpy()

#### Prepare target model

Now, we need to initialize our pre-trained Lightweight Deep Convolutional Neural Network (short DCTI) as a generic `Classifier`. Therefore we need to specify the loss function used to train the model (in our case the `tf.keras.losses.CategoricalCrossentropy`), the number of classes and the input shape of our CIFAR-10 dataset.

In [ ]:
# Initalize TensorFlow target model
target_model = Classifier(
    tf_dcti.load_dcti(), # TensorFlow DCTI
    loss=tf.keras.losses.CategoricalCrossentropy(), # Loss function of the TensorFlow target model
    nb_classes=TFCIFAR10.N_CLASSES, # Number of classes of the CIFAR10 dataset
    input_shape=TFCIFAR10.INPUT_SHAPE # Input shape of the CIFAR10 dataset
)

#### Perpare attack analysis

Next, we prepare our attack analysis. To initialize our attack analysis we define the Membership Inference Attack method we want to perform (in this case we use the `MembershipInferenceBlackBoxRuleBasedAttack`) and the Attack Input Data. The Attack Input Data consists of two different sets. The first contains the data (`x_train`) and its corresponding labels (`y_train`) which were used to train the target model. The second contains the data (`x_test`) and its corresponding labels (`y_test`) which were not part of the training process of the target model.

In [ ]:
attack_analysis = MembershipInferenceAttackAnalysis(
    MembershipInferenceBlackBoxRuleBasedAttack, 
    AttackInputData(
        x_train[:100], 
        y_train[:100], 
        x_test[:100], 
        y_test[:100]
    )
)

#### Define the slicing

Now we can define the slicing for our analysis. The slicing defines how the data will be sliced. Each slice will then be analysed separately. 

In [ ]:
slicing = Slicing(
    entire_dataset=True, 
    by_class=True, 
    by_classification_correctness=True
)

#### Perform Membership Inference Attack Analysis

Finally, we can perform our Membership Inference Attack Analysis. Therefore, we input the target model, the data that should be analysed, the membership labels (i.e. the labels which correctly describe if a data point is a member of the training dataset or not) and the splicing specification into the `analyse()` method. As a result, we get for each slice the indices of the corresponding data points, a human-readable description of the slice and the advantage score of the Membership Inference Attack (for more details about the advantage score, please read the following paper: https://arxiv.org/abs/1709.01604).

In [ ]:
result = attack_analysis.analyse(
    target_model, 
    np.concatenate((x_train[:100], x_test[:100])), 
    np.concatenate((y_train[:100], y_test[:100])), 
    np.concatenate((np.ones(len(x_train[:100])), np.zeros(len(x_test[:100])))), 
    slicing
)

print("\n".join((str(r) for r in result)))